In [10]:
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [19]:
def pad_sequences(data_list, max_length=None, pad_value=-100.0):
    """
    Pad variable length sequences to the same length
    
    Args:
        data_list: List of tensors with different sequence lengths
        max_length: Maximum length to pad to (default: longest sequence)
        pad_value: Value to use for padding
    
    Returns:
        padded_tensor: [batch_size, max_length, ...] - padded sequences
        seq_lengths: [batch_size] - original sequence lengths
    """
    if max_length is None:
        max_length = max(data.shape[0] for data in data_list) # Auto-calculate the max length

    batch_size = len(data_list) # Batch size
    seq_lengths = torch.tensor([data.shape[0] for data in data_list]) # Actual sequential length for each experiments
    dimensions = data_list[0].shape[1:] # Get shape of individual elements
    padded_tensor = torch.full((batch_size, max_length) + dimensions, pad_value, dtype=torch.float32) # generaste padded tensor filled with pad_value


    # Fill with actual data
    for i, data in enumerate(data_list):
        padded_tensor[i, :data.shape[0]] = torch.tensor(data[:data.shape[0]], dtype=torch.float32)
    
    return padded_tensor, seq_lengths, max_length

In [12]:
def prepare_input_data(Vt, E, C, Vm, I, seq_lengths=None):
    """
    Prepare input data for the model with padding support
    
    Args:
        voltage: [batch_size, seq_len] - applied voltage
        ext_electrolyte: [batch_size, seq_len] - external electrolyte concentration
        concentrations: [batch_size, seq_len, 3, 2] - [Feed, Acid, Base] x [LA, K] concentrations
        volumes: [batch_size, seq_len, 3] - volumes for each channel
        currents: [batch_size, seq_len] - measured currents
        seq_lengths: [batch_size] - actual sequence lengths (optional)
    
    Returns:
        input_tensor: [batch_size, seq_len, 3, 6] - formatted input for CNN-LSTM
        initial_state: [batch_size, 3, 3] - initial concentrations and volumes
        mask: [batch_size, seq_len] - padding mask
        seq_lengths: [batch_size] - actual sequence lengths
    """
    batch_size, seq_len = Vt.shape # Get batch size and sequence length for set the size of input tensor
    input = torch.zeros(batch_size, seq_len, 3, 6) # Generate input tensor

    # Fill input tensor for each channel
    for channel in range(3):
        input[:, :, channel, 0] = C[:, :, channel, 0] # LA concentration
        input[:, :, channel, 1] = C[:, :, channel, 1] # K concentration
        input[:, :, channel, 2] = Vm[:, :, channel] # Volume
        input[:, :, channel, 3] = Vt # Voltage (same for all)
        input[:, :, channel, 4] = E # Ext electrolyte (same for all)
        input[:, :, channel, 5] = I # Current (same for all)

    # Initial State for each channel
    init = torch.zeros(batch_size,3,3)
    init[:, :, 0] = C[:, 0, :, 0] # Initial LA concentrations [batch number, sequence length = 0 = initial state, channel number, feature]
    init[:, :, 1] = C[:, 0, :, 1] # Initial K concentrations
    init[:, :, 2] = Vm[:, 0, :] # Initial volumes

    # Create padding mask
    mask = torch.zeros(batch_size, seq_len)
    for i, length in enumerate(seq_lengths):
        mask[i, :length] = 1.0
    
    return input, init, mask, seq_lengths

In [13]:
df = pd.read_csv("BMED_DB_augmented.csv")
df['CA_K'] = 0.0
df['CB_LA'] = 0.0

In [16]:
# Input Data Preparation
Vt_list = []
E_list = []
C_list = []
Vm_list = []
I_list = []

for exp_num in df['exp'].unique():
    Vt_list.append(df[df['exp']==exp_num]['V'].values)
    E_list.append(df[df['exp']==exp_num]['E'].values)
    
    # CF, CA, CB 순으로 묶고 LA, K 순으로 값을 저장
    CF_LA = df[df['exp']==exp_num]['CF_LA'].values
    CF_K = df[df['exp']==exp_num]['CF_K'].values
    CA_LA = df[df['exp']==exp_num]['CA_LA'].values
    CA_K = df[df['exp']==exp_num]['CA_K'].values
    CB_LA = df[df['exp']==exp_num]['CB_LA'].values
    CB_K = df[df['exp']==exp_num]['CB_K'].values
    
    # 시간순으로 데이터 정렬
    C_exp = np.stack([
        np.stack([CF_LA, CF_K], axis=1),  # Feed (LA, K)
        np.stack([CA_LA, CA_K], axis=1),  # Acid (LA, K)
        np.stack([CB_LA, CB_K], axis=1)   # Base (LA, K)
    ], axis=1)
    
    C_list.append(C_exp)
    Vm_list.append(df[df['exp']==exp_num][['VF', 'VA', 'VB']].values)
    I_list.append(df[df['exp']==exp_num]['I'].values)


In [20]:
Vt, seq_lengths, max_length = pad_sequences(Vt_list)
E, _, _ = pad_sequences(E_list)
C, _, _ = pad_sequences(C_list)
Vm, _, _ = pad_sequences(Vm_list)
I, _, _ = pad_sequences(I_list)

In [22]:
# Prepare inputs
input_tensor, initial_state, mask, seq_lengths = prepare_input_data(Vt, E, C, Vm, I, seq_lengths)

In [8]:
class BMEDModel(nn.Module):
    def __init__(self, hidden_nodes = 64, rnn_layers = 2, cnn_channels = 32, max_seq_len = 37):
        super(BMEDModel, self).__init__()

        

tensor([[[[ 5.0000e-01,  1.0000e+00,  1.0000e+00,  2.0000e+01,  2.5000e-01,
            0.0000e+00],
          [ 0.0000e+00,  0.0000e+00,  1.0000e+00,  2.0000e+01,  2.5000e-01,
            0.0000e+00],
          [ 0.0000e+00,  0.0000e+00,  1.0000e+00,  2.0000e+01,  2.5000e-01,
            0.0000e+00]],

         [[ 4.9667e-01,  1.0218e+00,  1.0017e+00,  2.0000e+01,  2.5000e-01,
            2.5000e-02],
          [ 2.0328e-03,  0.0000e+00,  9.9889e-01,  2.0000e+01,  2.5000e-01,
            2.5000e-02],
          [ 0.0000e+00, -2.4251e-02,  9.9917e-01,  2.0000e+01,  2.5000e-01,
            2.5000e-02]],

         [[ 4.9335e-01,  1.0370e+00,  1.0062e+00,  2.0000e+01,  2.5000e-01,
            5.0000e-02],
          [ 3.0767e-03,  0.0000e+00,  9.9662e-01,  2.0000e+01,  2.5000e-01,
            5.0000e-02],
          [ 0.0000e+00, -4.5457e-02,  9.9697e-01,  2.0000e+01,  2.5000e-01,
            5.0000e-02]],

         ...,

         [[-1.0000e+02, -1.0000e+02, -1.0000e+02, -1.0000e+02, -1.0000

In [1]:
import torch
import torch.nn as nn
import numpy as np

class BMEDModel(nn.Module):
    def __init__(self, hidden_size=64, rnn_layers=2, cnn_channels=32, max_seq_len=28, rnn_type='LSTM'):
        super(BMEDModel, self).__init__()
        
        # Input dimensions
        # 3 channels (Feed, Acid, Base) x 6 features each
        # Features per channel: [LA_conc, K_conc, Volume, Voltage, Ext_electrolyte, Current]
        self.input_channels = 3
        self.features_per_channel = 6
        self.max_seq_len = max_seq_len
        self.rnn_type = rnn_type
        
        # CNN layers for spatial (channel-wise) feature extraction
        self.cnn_layers = nn.Sequential(
            nn.Conv1d(self.features_per_channel, cnn_channels, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv1d(cnn_channels, cnn_channels//2, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.AdaptiveAvgPool1d(1)  # Global average pooling
        )
        
        # RNN layers for temporal dependency (LSTM, RNN, or GRU)
        if rnn_type.upper() == 'LSTM':
            self.rnn = nn.LSTM(
                input_size=cnn_channels//2, 
                hidden_size=hidden_size, 
                num_layers=rnn_layers, 
                batch_first=True,
                dropout=0.2 if rnn_layers > 1 else 0
            )
        elif rnn_type.upper() == 'GRU':
            self.rnn = nn.GRU(
                input_size=cnn_channels//2, 
                hidden_size=hidden_size, 
                num_layers=rnn_layers, 
                batch_first=True,
                dropout=0.2 if rnn_layers > 1 else 0
            )
        elif rnn_type.upper() == 'RNN':
            self.rnn = nn.RNN(
                input_size=cnn_channels//2, 
                hidden_size=hidden_size, 
                num_layers=rnn_layers, 
                batch_first=True,
                dropout=0.2 if rnn_layers > 1 else 0,
                nonlinearity='tanh'
            )
        else:
            raise ValueError(f"Unsupported RNN type: {rnn_type}. Choose from 'LSTM', 'GRU', or 'RNN'.")
        
        # Output layers for flux prediction
        self.flux_predictor = nn.Sequential(
            nn.Linear(hidden_size, hidden_size//2),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(hidden_size//2, 4)  # [Water_flux, LA_flux, K_flux, Current]
        )
        
        # Physical layer for mass balance
        self.physical_layer = PhysicalLayer()
    
    def forward(self, x, initial_state, seq_lengths=None, mask=None):
        """
        x: [batch_size, sequence_length, channels, features]
        initial_state: [batch_size, channels, features] - initial concentrations and volumes
        seq_lengths: [batch_size] - actual sequence lengths for each sample
        mask: [batch_size, sequence_length] - mask for padded positions
        """
        batch_size, seq_len, channels, features = x.shape
        
        # Reshape for CNN processing
        x_cnn = x.view(batch_size * seq_len, features, channels)
        
        # CNN feature extraction
        cnn_features = self.cnn_layers(x_cnn)  # [batch*seq, channels//2, 1]
        cnn_features = cnn_features.squeeze(-1)  # [batch*seq, channels//2]
        
        # Reshape back for RNN
        rnn_input = cnn_features.view(batch_size, seq_len, -1)
        
        # Pack padded sequence for RNN if sequence lengths are provided
        if seq_lengths is not None:
            rnn_input = nn.utils.rnn.pack_padded_sequence(
                rnn_input, seq_lengths.cpu(), batch_first=True, enforce_sorted=False
            )
            rnn_out, _ = self.rnn(rnn_input)
            rnn_out, _ = nn.utils.rnn.pad_packed_sequence(rnn_out, batch_first=True)
        else:
            rnn_out, _ = self.rnn(rnn_input)
        
        # Predict fluxes for each time step
        fluxes = self.flux_predictor(rnn_out)  # [batch, seq_len, 4]
        
        # Apply mask to fluxes if provided
        if mask is not None:
            fluxes = fluxes * mask.unsqueeze(-1)
        
        # Apply physical layer to calculate concentrations and volumes
        outputs = self.physical_layer(fluxes, initial_state, seq_lengths, mask)
        
        return outputs, fluxes

class PhysicalLayer(nn.Module):
    def __init__(self, dt=0.5):  # 0.5 hour time step
        super(PhysicalLayer, self).__init__()
        self.dt = dt
    
    def forward(self, fluxes, initial_state, seq_lengths=None, mask=None):
        """
        fluxes: [batch_size, seq_len, 4] - [Water_flux, LA_flux, K_flux, Current]
        initial_state: [batch_size, 3, 3] - [Feed, Acid, Base] x [LA_conc, K_conc, Volume]
        seq_lengths: [batch_size] - actual sequence lengths for each sample
        mask: [batch_size, seq_len] - mask for padded positions
        
        Returns: [batch_size, seq_len, 3, 3] - time series of [LA_conc, K_conc, Volume] for each channel
        """
        batch_size, seq_len, _ = fluxes.shape
        
        # Initialize output tensor
        outputs = torch.zeros(batch_size, seq_len, 3, 3)  # [batch, time, channel, property]
        
        # Set initial conditions
        current_state = initial_state.clone()  # [batch, channel, property]
        
        for t in range(seq_len):
            # Extract fluxes for current time step
            water_flux = fluxes[:, t, 0]  # Water movement
            la_flux = fluxes[:, t, 1]     # LA movement (Feed → Acid)
            k_flux = fluxes[:, t, 2]      # K movement (Feed → Base)
            
            # Only update if within actual sequence length
            if seq_lengths is not None:
                # Create mask for this time step
                time_mask = (t < seq_lengths).float()
                water_flux = water_flux * time_mask
                la_flux = la_flux * time_mask
                k_flux = k_flux * time_mask
            elif mask is not None:
                time_mask = mask[:, t]
                water_flux = water_flux * time_mask
                la_flux = la_flux * time_mask
                k_flux = k_flux * time_mask
            
            # Mass balance calculations
            current_state = self.mass_balance_step(current_state, water_flux, la_flux, k_flux)
            
            # Store results
            outputs[:, t, :, :] = current_state
        
        return outputs
    
    def mass_balance_step(self, state, water_flux, la_flux, k_flux):
        """
        Perform one time step of mass balance
        state: [batch, 3, 3] - [Feed, Acid, Base] x [LA_conc, K_conc, Volume]
        """
        batch_size = state.shape[0]
        new_state = state.clone()
        
        # Extract current values
        # Channel 0: Feed, Channel 1: Acid, Channel 2: Base
        # Property 0: LA_conc, Property 1: K_conc, Property 2: Volume
        
        feed_la_conc = state[:, 0, 0]
        feed_k_conc = state[:, 0, 1]
        feed_vol = state[:, 0, 2]
        
        acid_la_conc = state[:, 1, 0]
        acid_vol = state[:, 1, 2]
        
        base_k_conc = state[:, 2, 1]
        base_vol = state[:, 2, 2]
        
        # Volume changes due to water flux
        # Assuming positive flux means water moves from feed
        new_state[:, 0, 2] = feed_vol - water_flux * self.dt  # Feed volume decreases
        new_state[:, 1, 2] = acid_vol + water_flux * self.dt * 0.5  # Acid volume increases
        new_state[:, 2, 2] = base_vol + water_flux * self.dt * 0.5  # Base volume increases
        
        # LA mass balance (Feed → Acid)
        la_moles_transferred = la_flux * self.dt
        
        # Feed LA decreases
        feed_la_moles = feed_la_conc * feed_vol
        new_feed_la_moles = torch.clamp(feed_la_moles - la_moles_transferred, min=0)
        new_state[:, 0, 0] = new_feed_la_moles / torch.clamp(new_state[:, 0, 2], min=1e-6)
        
        # Acid LA increases
        acid_la_moles = acid_la_conc * acid_vol
        new_acid_la_moles = acid_la_moles + la_moles_transferred
        new_state[:, 1, 0] = new_acid_la_moles / torch.clamp(new_state[:, 1, 2], min=1e-6)
        
        # K mass balance (Feed → Base)
        k_moles_transferred = k_flux * self.dt
        
        # Feed K decreases
        feed_k_moles = feed_k_conc * feed_vol
        new_feed_k_moles = torch.clamp(feed_k_moles - k_moles_transferred, min=0)
        new_state[:, 0, 1] = new_feed_k_moles / torch.clamp(new_state[:, 0, 2], min=1e-6)
        
        # Base K increases
        base_k_moles = base_k_conc * base_vol
        new_base_k_moles = base_k_moles + k_moles_transferred
        new_state[:, 2, 1] = new_base_k_moles / torch.clamp(new_state[:, 2, 2], min=1e-6)
        
        # Acid and Base don't have K and LA respectively (constraint)
        new_state[:, 1, 1] = 0  # No K in acid
        new_state[:, 2, 0] = 0  # No LA in base
        
        return new_state

def prepare_input_data(voltage, ext_electrolyte, concentrations, volumes, currents, seq_lengths=None):
    """
    Prepare input data for the model with padding support
    
    Args:
        voltage: [batch_size, seq_len] - applied voltage
        ext_electrolyte: [batch_size, seq_len] - external electrolyte concentration
        concentrations: [batch_size, seq_len, 3, 2] - [Feed, Acid, Base] x [LA, K] concentrations
        volumes: [batch_size, seq_len, 3] - volumes for each channel
        currents: [batch_size, seq_len] - measured currents
        seq_lengths: [batch_size] - actual sequence lengths (optional)
    
    Returns:
        input_tensor: [batch_size, seq_len, 3, 6] - formatted input for CNN-LSTM
        initial_state: [batch_size, 3, 3] - initial concentrations and volumes
        mask: [batch_size, seq_len] - padding mask
        seq_lengths: [batch_size] - actual sequence lengths
    """
    batch_size, seq_len = voltage.shape
    
    # Create input tensor
    input_tensor = torch.zeros(batch_size, seq_len, 3, 6)
    
    # Fill input tensor for each channel
    for channel in range(3):
        input_tensor[:, :, channel, 0] = concentrations[:, :, channel, 0]  # LA concentration
        input_tensor[:, :, channel, 1] = concentrations[:, :, channel, 1]  # K concentration
        input_tensor[:, :, channel, 2] = volumes[:, :, channel]            # Volume
        input_tensor[:, :, channel, 3] = voltage                          # Voltage (same for all)
        input_tensor[:, :, channel, 4] = ext_electrolyte                  # Ext electrolyte (same for all)
        input_tensor[:, :, channel, 5] = currents                        # Current (same for all)
    
    # Initial state: [LA_conc, K_conc, Volume] for each channel at t=0
    initial_state = torch.zeros(batch_size, 3, 3)
    initial_state[:, :, 0] = concentrations[:, 0, :, 0]  # Initial LA concentrations
    initial_state[:, :, 1] = concentrations[:, 0, :, 1]  # Initial K concentrations
    initial_state[:, :, 2] = volumes[:, 0, :]            # Initial volumes
    
    # Create padding mask
    if seq_lengths is None:
        seq_lengths = torch.full((batch_size,), seq_len, dtype=torch.long)
    
    mask = torch.zeros(batch_size, seq_len)
    for i, length in enumerate(seq_lengths):
        mask[i, :length] = 1.0
    
    return input_tensor, initial_state, mask, seq_lengths

def pad_sequences(data_list, max_length=None, pad_value=0.0):
    """
    Pad variable length sequences to the same length
    
    Args:
        data_list: List of tensors with different sequence lengths
        max_length: Maximum length to pad to (default: longest sequence)
        pad_value: Value to use for padding
    
    Returns:
        padded_tensor: [batch_size, max_length, ...] - padded sequences
        seq_lengths: [batch_size] - original sequence lengths
    """
    if max_length is None:
        max_length = max(data.shape[0] for data in data_list)
    
    batch_size = len(data_list)
    seq_lengths = torch.tensor([data.shape[0] for data in data_list])
    
    # Get shape of individual elements
    example_shape = data_list[0].shape[1:]  # Remove time dimension
    
    # Create padded tensor
    padded_tensor = torch.full(
        (batch_size, max_length) + example_shape, 
        pad_value, 
        dtype=data_list[0].dtype
    )
    
    # Fill with actual data
    for i, data in enumerate(data_list):
        actual_length = min(data.shape[0], max_length)
        padded_tensor[i, :actual_length] = data[:actual_length]
    
    return padded_tensor, seq_lengths

# Example usage with variable length sequences
def create_model_with_padding_example():
    # Model instantiation with different RNN types
    print("Creating models with different RNN types...")
    
    models = {}
    models['LSTM'] = BMEDModel(hidden_size=64, rnn_layers=2, cnn_channels=32, max_seq_len=28, rnn_type='LSTM')
    models['RNN'] = BMEDModel(hidden_size=64, rnn_layers=2, cnn_channels=32, max_seq_len=28, rnn_type='RNN')
    models['GRU'] = BMEDModel(hidden_size=64, rnn_layers=2, cnn_channels=32, max_seq_len=28, rnn_type='GRU')
    
    # Example: Create variable length sequences
    batch_size = 4
    max_seq_len = 28  # Updated to 28 for 14 hours
    
    # Simulate different experiment durations
    actual_lengths = [28, 20, 25, 15]  # Different experiment durations
    
    # Create variable length data
    voltage_list = []
    ext_electrolyte_list = []
    concentrations_list = []
    volumes_list = []
    currents_list = []
    
    for length in actual_lengths:
        voltage_list.append(torch.randn(length) * 5 + 10)
        ext_electrolyte_list.append(torch.randn(length) * 0.1 + 0.5)
        concentrations_list.append(torch.randn(length, 3, 2) * 0.1 + 0.5)
        volumes_list.append(torch.randn(length, 3) * 0.05 + 1.0)
        currents_list.append(torch.randn(length) * 2 + 5)
    
    # Pad sequences
    voltage_padded, seq_lengths = pad_sequences(voltage_list, max_seq_len)
    ext_electrolyte_padded, _ = pad_sequences(ext_electrolyte_list, max_seq_len)
    concentrations_padded, _ = pad_sequences(concentrations_list, max_seq_len)
    volumes_padded, _ = pad_sequences(volumes_list, max_seq_len)
    currents_padded, _ = pad_sequences(currents_list, max_seq_len)
    
    # Prepare inputs
    input_tensor, initial_state, mask, seq_lengths = prepare_input_data(
        voltage_padded, ext_electrolyte_padded, concentrations_padded, 
        volumes_padded, currents_padded, seq_lengths
    )
    
    # Test all models
    results = {}
    for rnn_type, model in models.items():
        print(f"\n=== Testing {rnn_type} Model ===")
        
        # Forward pass with padding support
        outputs, fluxes = model(input_tensor, initial_state, seq_lengths, mask)
        
        print(f"Input shape: {input_tensor.shape}")
        print(f"Sequence lengths: {seq_lengths}")
        print(f"Output shape: {outputs.shape}")
        print(f"Fluxes shape: {fluxes.shape}")
        
        # Count parameters
        total_params = sum(p.numel() for p in model.parameters())
        print(f"Total parameters: {total_params:,}")
        
        results[rnn_type] = {
            'model': model,
            'outputs': outputs,
            'fluxes': fluxes,
            'params': total_params
        }
    
    # Show parameter comparison
    print(f"\n=== Parameter Comparison ===")
    for rnn_type, result in results.items():
        print(f"{rnn_type}: {result['params']:,} parameters")
    
    return results, input_tensor, initial_state, mask, seq_lengths

# Example usage and training setup
def create_model_and_example():
    # Model instantiation with LSTM (default)
    model = BMEDModel(hidden_size=64, rnn_layers=2, cnn_channels=32, rnn_type='LSTM')
    
    # Example data shapes
    batch_size = 16
    seq_len = 28  # 14 hours / 0.5 hour steps
    
    # Example input preparation
    voltage = torch.randn(batch_size, seq_len) * 5 + 10  # 10±5V
    ext_electrolyte = torch.randn(batch_size, seq_len) * 0.1 + 0.5  # 0.5±0.1 M
    
    # Concentrations [batch, time, channel, component]
    concentrations = torch.randn(batch_size, seq_len, 3, 2) * 0.1 + 0.5
    volumes = torch.randn(batch_size, seq_len, 3) * 0.05 + 1.0  # 1±0.05 L
    currents = torch.randn(batch_size, seq_len) * 2 + 5  # 5±2 A
    
    # Prepare inputs
    input_tensor, initial_state, mask, seq_lengths = prepare_input_data(
        voltage, ext_electrolyte, concentrations, volumes, currents
    )
    
    # Forward pass
    outputs, fluxes = model(input_tensor, initial_state)
    
    print(f"Input shape: {input_tensor.shape}")
    print(f"Initial state shape: {initial_state.shape}")
    print(f"Output shape: {outputs.shape}")  # [batch, seq_len, 3, 3]
    print(f"Fluxes shape: {fluxes.shape}")   # [batch, seq_len, 4]
    
    return model, input_tensor, initial_state, outputs, fluxes

if __name__ == "__main__":
    # Example with fixed length sequences
    print("=== Fixed Length Example ===")
    model, inputs, initial_state, outputs, fluxes = create_model_and_example()
    print("Fixed length model created successfully!\n")
    
    # Example with variable length sequences and different RNN types
    print("=== Variable Length (Padded) Example with Different RNN Types ===")
    results, inputs_padded, initial_state_padded, mask, seq_lengths = create_model_with_padding_example()
    print("All models created successfully!")
    
    # Example of loss calculation with masking
    print("\n=== Loss Calculation with Masking ===")
    
    # Use LSTM model for loss calculation example
    lstm_outputs = results['LSTM']['outputs']
    lstm_fluxes = results['LSTM']['fluxes']
    
    # Dummy target data
    target_outputs = torch.randn_like(lstm_outputs)
    target_fluxes = torch.randn_like(lstm_fluxes)
    
    # Masked loss calculation
    def masked_mse_loss(predictions, targets, mask):
        """Calculate MSE loss only for non-padded positions"""
        loss = ((predictions - targets) ** 2) * mask.unsqueeze(-1)
        return loss.sum() / mask.sum()
    
    # Calculate losses for LSTM
    output_loss = masked_mse_loss(lstm_outputs.view(lstm_outputs.shape[0], lstm_outputs.shape[1], -1), 
                                 target_outputs.view(target_outputs.shape[0], target_outputs.shape[1], -1), 
                                 mask)
    flux_loss = masked_mse_loss(lstm_fluxes, target_fluxes, mask)
    
    print(f"LSTM Output loss (masked): {output_loss.item():.4f}")
    print(f"LSTM Flux loss (masked): {flux_loss.item():.4f}")
    print(f"LSTM Total loss: {(output_loss + flux_loss).item():.4f}")
    
    # Speed comparison (rough estimate)
    print(f"\n=== Speed and Complexity Comparison ===")
    print("RNN Type | Parameters | Relative Speed | Best For")
    print("-" * 55)
    print("RNN      | Least      | Fastest        | Simple patterns, short sequences")
    print("GRU      | Medium     | Medium         | Balance of performance & speed")
    print("LSTM     | Most       | Slowest        | Complex patterns, long sequences")
    print("\nFor your 28-step BMED system, LSTM is recommended for best performance.")

=== Fixed Length Example ===
Input shape: torch.Size([16, 28, 3, 6])
Initial state shape: torch.Size([16, 3, 3])
Output shape: torch.Size([16, 28, 3, 3])
Fluxes shape: torch.Size([16, 28, 4])
Fixed length model created successfully!

=== Variable Length (Padded) Example with Different RNN Types ===
Creating models with different RNN types...

=== Testing LSTM Model ===
Input shape: torch.Size([4, 28, 3, 6])
Sequence lengths: tensor([28, 20, 25, 15])
Output shape: torch.Size([4, 28, 3, 3])
Fluxes shape: torch.Size([4, 28, 4])
Total parameters: 58,644

=== Testing RNN Model ===
Input shape: torch.Size([4, 28, 3, 6])
Sequence lengths: tensor([28, 20, 25, 15])
Output shape: torch.Size([4, 28, 3, 3])
Fluxes shape: torch.Size([4, 28, 4])
Total parameters: 17,940

=== Testing GRU Model ===
Input shape: torch.Size([4, 28, 3, 6])
Sequence lengths: tensor([28, 20, 25, 15])
Output shape: torch.Size([4, 28, 3, 3])
Fluxes shape: torch.Size([4, 28, 4])
Total parameters: 45,076

=== Parameter Compari